# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 20 2022 2:36PM,246862,10,MSP214137,"Methapharm, Inc.",Released
1,Sep 20 2022 2:36PM,246862,10,MSP214139,"Methapharm, Inc.",Released
2,Sep 20 2022 2:36PM,246862,10,MSP214140,"Methapharm, Inc.",Released
3,Sep 20 2022 2:36PM,246862,10,MSP214142,"Methapharm, Inc.",Released
4,Sep 20 2022 2:24PM,246861,10,8551545,Bio-PRF,Released
5,Sep 20 2022 2:04PM,246860,10,8526765-BO,"ACI Healthcare USA, Inc.",Released
6,Sep 20 2022 2:03PM,246859,12,8551533,LF of America Corp.,Released
7,Sep 20 2022 2:02PM,246858,10,8526764,"ACI Healthcare USA, Inc.",Released
8,Sep 20 2022 1:56PM,246856,16,SHL-8551532,"SHL Pharma, LLC",Released
9,Sep 20 2022 1:56PM,246857,10,8431946-BO,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
37,246858,Released,1
38,246859,Released,1
39,246860,Released,1
40,246861,Released,1
41,246862,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246858,NaN,NaN,1.0
246859,NaN,NaN,1.0
246860,NaN,NaN,1.0
246861,NaN,NaN,1.0
246862,NaN,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246711,0.0,27.0,5.0
246762,0.0,0.0,1.0
246770,0.0,1.0,0.0
246785,0.0,1.0,0.0
246789,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246711,0,27,5
246762,0,0,1
246770,0,1,0
246785,0,1,0
246789,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246711,0,27,5
1,246762,0,0,1
2,246770,0,1,0
3,246785,0,1,0
4,246789,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246711,,27,5
1,246762,,,1
2,246770,,1,
3,246785,,1,
4,246789,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 20 2022 2:36PM,246862,10,"Methapharm, Inc."
4,Sep 20 2022 2:24PM,246861,10,Bio-PRF
5,Sep 20 2022 2:04PM,246860,10,"ACI Healthcare USA, Inc."
6,Sep 20 2022 2:03PM,246859,12,LF of America Corp.
7,Sep 20 2022 2:02PM,246858,10,"ACI Healthcare USA, Inc."
8,Sep 20 2022 1:56PM,246856,16,"SHL Pharma, LLC"
9,Sep 20 2022 1:56PM,246857,10,"ACI Healthcare USA, Inc."
12,Sep 20 2022 1:40PM,246853,19,"AdvaGen Pharma, Ltd"
18,Sep 20 2022 1:29PM,246850,22,"NBTY Global, Inc."
19,Sep 20 2022 1:28PM,246849,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 20 2022 2:36PM,246862,10,"Methapharm, Inc.",,,4
1,Sep 20 2022 2:24PM,246861,10,Bio-PRF,,,1
2,Sep 20 2022 2:04PM,246860,10,"ACI Healthcare USA, Inc.",,,1
3,Sep 20 2022 2:03PM,246859,12,LF of America Corp.,,,1
4,Sep 20 2022 2:02PM,246858,10,"ACI Healthcare USA, Inc.",,,1
5,Sep 20 2022 1:56PM,246856,16,"SHL Pharma, LLC",,,1
6,Sep 20 2022 1:56PM,246857,10,"ACI Healthcare USA, Inc.",,,3
7,Sep 20 2022 1:40PM,246853,19,"AdvaGen Pharma, Ltd",,,6
8,Sep 20 2022 1:29PM,246850,22,"NBTY Global, Inc.",,,1
9,Sep 20 2022 1:28PM,246849,22,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 20 2022 2:36PM,246862,10,"Methapharm, Inc.",4,,
1,Sep 20 2022 2:24PM,246861,10,Bio-PRF,1,,
2,Sep 20 2022 2:04PM,246860,10,"ACI Healthcare USA, Inc.",1,,
3,Sep 20 2022 2:03PM,246859,12,LF of America Corp.,1,,
4,Sep 20 2022 2:02PM,246858,10,"ACI Healthcare USA, Inc.",1,,
5,Sep 20 2022 1:56PM,246856,16,"SHL Pharma, LLC",1,,
6,Sep 20 2022 1:56PM,246857,10,"ACI Healthcare USA, Inc.",3,,
7,Sep 20 2022 1:40PM,246853,19,"AdvaGen Pharma, Ltd",6,,
8,Sep 20 2022 1:29PM,246850,22,"NBTY Global, Inc.",1,,
9,Sep 20 2022 1:28PM,246849,22,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 20 2022 2:36PM,246862,10,"Methapharm, Inc.",4,,
1,Sep 20 2022 2:24PM,246861,10,Bio-PRF,1,,
2,Sep 20 2022 2:04PM,246860,10,"ACI Healthcare USA, Inc.",1,,
3,Sep 20 2022 2:03PM,246859,12,LF of America Corp.,1,,
4,Sep 20 2022 2:02PM,246858,10,"ACI Healthcare USA, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 20 2022 2:36PM,246862,10,"Methapharm, Inc.",4.0,NaN,NaN
1,Sep 20 2022 2:24PM,246861,10,Bio-PRF,1.0,NaN,NaN
2,Sep 20 2022 2:04PM,246860,10,"ACI Healthcare USA, Inc.",1.0,NaN,NaN
3,Sep 20 2022 2:03PM,246859,12,LF of America Corp.,1.0,NaN,NaN
4,Sep 20 2022 2:02PM,246858,10,"ACI Healthcare USA, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 20 2022 2:36PM,246862,10,"Methapharm, Inc.",4.0,0.0,0.0
1,Sep 20 2022 2:24PM,246861,10,Bio-PRF,1.0,0.0,0.0
2,Sep 20 2022 2:04PM,246860,10,"ACI Healthcare USA, Inc.",1.0,0.0,0.0
3,Sep 20 2022 2:03PM,246859,12,LF of America Corp.,1.0,0.0,0.0
4,Sep 20 2022 2:02PM,246858,10,"ACI Healthcare USA, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2962099,74.0,0.0,0.0
12,246859,1.0,0.0,0.0
15,740334,23.0,66.0,18.0
16,1974664,18.0,10.0,0.0
19,493688,12.0,12.0,4.0
20,740419,61.0,28.0,12.0
21,246770,0.0,1.0,0.0
22,493699,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2962099,74.0,0.0,0.0
1,12,246859,1.0,0.0,0.0
2,15,740334,23.0,66.0,18.0
3,16,1974664,18.0,10.0,0.0
4,19,493688,12.0,12.0,4.0
5,20,740419,61.0,28.0,12.0
6,21,246770,0.0,1.0,0.0
7,22,493699,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,74.0,0.0,0.0
1,12,1.0,0.0,0.0
2,15,23.0,66.0,18.0
3,16,18.0,10.0,0.0
4,19,12.0,12.0,4.0
5,20,61.0,28.0,12.0
6,21,0.0,1.0,0.0
7,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,74.0
1,12,Released,1.0
2,15,Released,23.0
3,16,Released,18.0
4,19,Released,12.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,18.0,0.0,4.0,12.0,0.0,0.0
Executing,0.0,0.0,66.0,10.0,12.0,28.0,1.0,0.0
Released,74.0,1.0,23.0,18.0,12.0,61.0,0.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,18.0,0.0,4.0,12.0,0.0,0.0
1,Executing,0.0,0.0,66.0,10.0,12.0,28.0,1.0,0.0
2,Released,74.0,1.0,23.0,18.0,12.0,61.0,0.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,18.0,0.0,4.0,12.0,0.0,0.0
1,Executing,0.0,0.0,66.0,10.0,12.0,28.0,1.0,0.0
2,Released,74.0,1.0,23.0,18.0,12.0,61.0,0.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()